In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EvalPrediction
from datasets import load_dataset
from peft import DiagConfig, get_peft_model
import numpy as np
import torch
import evaluate

In [3]:
torch.set_printoptions(threshold=torch.inf)  # Display all elements
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.device_count()

1

In [4]:
BASE_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tok  = AutoTokenizer.from_pretrained(BASE_ID, use_fast=True)

base_model = AutoModelForSequenceClassification.from_pretrained(
        BASE_ID,
        num_labels=2,
        # load_in_4bit=True,
        device_map="auto")

diag_cfg = DiagConfig(
        target_modules=["q_proj","v_proj"],
        diag_alpha=1.0,
        diag_dropout=0.0,
        fan_in_fan_out=False,
        bias="none",
        init_diag_weights=True)
model = get_peft_model(base_model, diag_cfg)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[DiagConfig] Initializing configuration
[DiagConfig] Target modules: ['q_proj', 'v_proj']
[DiagConfig] Alpha: 1.0, Dropout: 0.0
[DiagConfig] Fan in/out: False, Bias: none
[DiagConfig] Init weights: True
[DiagModel] Creating/replacing layer q_proj with base layer type: Linear
[DiagModel] Base layer weight shape: torch.Size([2048, 2048])
[DiagModel] Creating new DiagLayer for q_proj
[DiagLayer] Initializing with base layer: Linear
[DiagLayer] Feature dims  – in: 2048, out: 2048
[DiagLayer] Updating layer for adapter: default
[DiagLayer] Creating new row weight for adapter: default
[DiagLayer] Created row weight with shape: torch.Size([2048, 2048]), dtype: torch.float32
[DiagLayer] Added adapter default to active adapters
[DiagModel] Creating/replacing layer v_proj with base layer type: Linear
[DiagModel] Base layer weight shape: torch.Size([256, 2048])
[DiagModel] Creating new DiagLayer for v_proj
[DiagLayer] Initializing with base layer: Linear
[DiagLayer] Feature dims  – in: 2048, out:

In [5]:
core_model   = model.get_base_model()        # → LlamaForSequenceClassification
llama_blocks = core_model.model.layers       # → ModuleList of decoder layers
qproj_0      = llama_blocks[0].self_attn.q_proj

print(type(qproj_0))          # should be your Linear4bit / Linear8bitLt
print(qproj_0.weight.shape)   # should be (out, in)  e.g.  (4096, 2048)


<class 'peft.tuners.diag.layer.Linear'>
torch.Size([2048, 2048])


In [6]:
model.config.pad_token_id = tok.pad_token_id

# ---------- data ----------
raw_ds = load_dataset("glue", "sst2")

def tokenize(batch):
    natural   = tok(batch["sentence"], add_special_tokens=True)
    true_lens = [len(ids) for ids in natural["input_ids"]]

    padded = tok(
        batch["sentence"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

    padded["real_length"] = true_lens
    return padded

tokenized_ds = raw_ds.map(
    tokenize,
    batched=True,
    remove_columns=["sentence", "idx"]
)

# rename + set Torch format
tokenized_ds = tokenized_ds.rename_column("label", "labels")
tokenized_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels", "real_length"],
)

# ---------- stats ----------
max_len = max(tokenized_ds["train"]["real_length"])
print(f"Longest raw sentence: {max_len} tokens")


# ---------- data ----------
raw_datasets = load_dataset("glue", "sst2")
def tokenize_function(example):
    return tok(example["sentence"], truncation=True, padding="max_length", max_length=100)

# Tokenize the entire dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
del raw_datasets
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Longest raw sentence: 74 tokens


In [7]:
# ---------- trainer ----------
args = TrainingArguments(
        output_dir="vera-tiny-sst2",
        per_device_train_batch_size=32,
        num_train_epochs=2,
        learning_rate=3e-3,
        eval_strategy="epoch",
        save_strategy="no",
        logging_steps=50)


trainer = Trainer(model=model,
                  args=args,
                  train_dataset=tokenized_datasets["train"],
                  eval_dataset=tokenized_datasets["validation"])

In [8]:
trainer.train()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [16]:
model.train()
with torch.amp.autocast("cuda"):  # updated syntax
    batch = tok(["hello"], return_tensors="pt").to(0)
    out = model(**batch, labels=torch.tensor([1]).to(0))
    print(out.loss.grad_fn)  # should NOT be None


None


In [17]:
print(type(out))
print(out)


<class 'transformers.modeling_outputs.SequenceClassifierOutputWithPast'>
SequenceClassifierOutputWithPast(loss=tensor(0.5234, device='cuda:0'), logits=tensor([[-1.2773, -0.9028]], device='cuda:0', dtype=torch.float16), past_key_values=None, hidden_states=None, attentions=None)


In [ ]:
model.train()
with torch.amp.autocast("cuda"):
    batch = tok(["hello"], return_tensors="pt").to(0)
    out = model(**batch, labels=torch.tensor([1]).to(0))

loss = out.loss.to(torch.float32)
loss.backward()


# Check grads manually
for name, param in model.named_parameters():
    if param.requires_grad and param.grad is not None:
        print(f"{name} has non-zero grad: {param.grad.abs().mean().item():.6f}")


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [21]:
model.train()
batch = tok(["hello"], return_tensors="pt").to(0)
out = model(**batch, labels=torch.tensor([1]).to(0))

print("loss requires grad?", out.loss.requires_grad)
print("loss grad_fn?", out.loss.grad_fn)


loss requires grad? False
loss grad_fn? None


In [ ]:
for n, p in model.named_parameters():
    if p.requires_grad:
        print(n, p.shape)


base_model.model.model.layers.0.self_attn.q_proj.row_weight.default torch.Size([2048, 2048])
base_model.model.model.layers.0.self_attn.v_proj.row_weight.default torch.Size([256, 2048])
base_model.model.model.layers.1.self_attn.q_proj.row_weight.default torch.Size([2048, 2048])
base_model.model.model.layers.1.self_attn.v_proj.row_weight.default torch.Size([256, 2048])
base_model.model.model.layers.2.self_attn.q_proj.row_weight.default torch.Size([2048, 2048])
base_model.model.model.layers.2.self_attn.v_proj.row_weight.default torch.Size([256, 2048])
base_model.model.model.layers.3.self_attn.q_proj.row_weight.default torch.Size([2048, 2048])
base_model.model.model.layers.3.self_attn.v_proj.row_weight.default torch.Size([256, 2048])
base_model.model.model.layers.4.self_attn.q_proj.row_weight.default torch.Size([2048, 2048])
base_model.model.model.layers.4.self_attn.v_proj.row_weight.default torch.Size([256, 2048])
base_model.model.model.layers.5.self_attn.q_proj.row_weight.default torch.S

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
logits = predictions.predictions[1]
preds = np.argmax(logits, axis=-1)
metric = evaluate.load("accuracy")

tokenized_datasets["validation"]["labels"]
metric.compute(predictions=preds, references=tokenized_datasets["validation"]["labels"])

In [ ]:
# For non trained model accuracy 0.4919
# For r=128 one epoch lr 3e-3 accuracy 0.932
# For r=128 two epochs lr 3e-3 accuracy 0.939

In [ ]:
# after trainer.train()
adapter_dir = "diag_adapter"
model.save_pretrained(adapter_dir, safe_serialization=True)  # adapter only
tok.save_pretrained(adapter_dir)                             # optional, for easy reload

In [ ]:
base = AutoModelForSequenceClassification.from_pretrained(
           BASE_ID, load_in_4bit=True, device_map="auto")
from peft import PeftModel
model = PeftModel.from_pretrained(base, "diag_adapter").to(device)